# Classical Portfolio Optimization

This notebook is used to generate a portfolio of 10 random stocks from the AlphaVantage API and then optimize the portfolio weights using Markowitz theory. The optimization analysis can be performed using 3 years of historical stock data (2014-2017). 

The returns of the portfolio with and without optimization over a 3 year test timeframe can then be calculated (2017-2020). These results will be used as the baseline in which the optimization performed through supervised and unsupervised learning will be compared.

In order to perform the optimization, the stock data must be obtained, the **expected returns** must be calculated, the **variance** of the stock data must be calculated and the **effecient frontier** must be found.


## Obtaining stock data



### Getting stock names
The stock names can be found by obtaining an array of all the stocks in the S&P 500 index, generating 10 random numbers and then using those numbers to choose 10 array items.

In [1]:
#Taken from Automating getting the S&P 500 list - Python Programming for Finance p.5
# https://pythonprogramming.net/sp500-company-list-python-programming-for-finance/

import bs4 as bs
import pickle
import requests

def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker.strip("\n"))
        
    with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers,f)
        
    return tickers

save_sp500_tickers()
    

['MMM',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADBE',
 'AMD',
 'AAP',
 'AES',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALXN',
 'ALGN',
 'ALLE',
 'AGN',
 'ADS',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'T',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'ANTM',
 'AON',
 'AOS',
 'APA',
 'AIV',
 'AAPL',
 'AMAT',
 'APTV',
 'ADM',
 'ARNC',
 'ANET',
 'AJG',
 'AIZ',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'BKR',
 'BLL',
 'BAC',
 'BK',
 'BAX',
 'BDX',
 'BRK.B',
 'BBY',
 'BIIB',
 'BLK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BF.B',
 'CHRW',
 'COG',
 'CDNS',
 'CPB',
 'COF',
 'CPRI',
 'CAH',
 'KMX',
 'CCL',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'CNC',
 'CNP',
 'CTL',
 'CERN',
 'CF',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CTXS',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CMA

Next 10 random numbers between 0 and 504 can be generated in order to pick 10 random stock tickers from the list

In [2]:
import random

tickers = save_sp500_tickers()
print(len(tickers)) # there are actually 505 stocks in the S&P 500, who knew!

def getRandomTickers(tickers):
  randIndex = []
  randTickers = []
  for i in range(0,10):
    randIndex.append(random.randint(0,504))
  for index in randIndex:
    randTickers.append(tickers[index])
  return randTickers

print(getRandomTickers(tickers)) 


505
['ROL', 'DD', 'LRCX', 'WRB', 'TRV', 'COTY', 'GD', 'NRG', 'ORCL', 'AMZN']


Now we can get the names of the tickers in the dataset. 

The data was extracted from the API using a NodeJS program which obtained daily price and volume data for each stock over a period of 20 years. Due to the failure to extract data for several stocks, the number of stock tickers is not exactly 505, but rather close to 500.

We first need to import the data from the csv into a pandas dataframe so that it can be further analyzed.

The data is stored in Google Drive and can be accessed via Colab after mounting to Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Now the csv can be imported into a dataframe


In [4]:
import pandas as pd
import time

#change directory to where the data is located 
# %cd drive/My Drive/cap4770-project
!pwd

#use timer to get process times
start_time = time.time()

#read csv
data = pd.read_csv("SP_DAILY_2000-2020.csv")
print("read data --- %s seconds ---" % (time.time() - start_time))
print()

#preview first 5 lines
start_time = time.time()
print(data.head())
print("get head --- %s seconds ---" % (time.time() - start_time))
print()

#preview last 5 lines
start_time = time.time()
print(data.tail())
print("get tail --- %s seconds ---" % (time.time() - start_time))
print()

#get size
start_time = time.time()
print("Size:", data.size)
print("get size --- %s seconds ---" % (time.time() - start_time))
print()

/content


FileNotFoundError: ignored

Markowitz Analysis requires a vector of expected returns over the period of analysis for all of the stocks in the portfolio as well as the cauculation of the covariance matrix for the stocks. This can then be used to generate the portfolio weight vector omega.

The ffn library will be used to retrieve data for the random tickers generated from yahoo finance from March 26, 2010 to March 26, 2015.Then the preliminary Markowitz analysis can be conducted


In [5]:
#setup
%matplotlib inline

#need to add these packages to requirements.txt
!pip install empyrical ffn PyPortfolioOpt

#taken from https://github.com/Poseyy/MarketAnalysis/blob/master/portfolios/PortfolioAnalysis.ipynb
import ffn 
from empyrical import alpha_beta
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import discrete_allocation
import matplotlib as pyplot
import numpy as np
import pandas as pd

/usr/local/lib/python3.6/dist-packages/ffn/core.py:27: MatplotlibDeprecationWarning: The 'warn' parameter of use() is deprecated since Matplotlib 3.1 and will be removed in 3.3.  If any parameter follows 'warn', they should be pass as keyword, not positionally.
  matplotlib.use('agg', warn=False)


In [0]:
#generate random portfolio
randomTickers = getRandomTickers(tickers)

In [0]:
#retrieve price data for training period, plot data

prices = ffn.get(randomTickers,start='2010-03-26', end='2015-03-26')
ax = prices.rebase().plot()

Now we can plot the returns for this portfolio

In [16]:
returns = prices.to_returns().dropna()
ax = returns.hist(figsize=(10,10))

/usr/local/lib/python3.6/dist-packages/pandas/plotting/_matplotlib/tools.py:307: MatplotlibDeprecationWarning: 
The rowNum attribute was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use ax.get_subplotspec().rowspan.start instead.
  layout[ax.rowNum, ax.colNum] = ax.get_visible()
/usr/local/lib/python3.6/dist-packages/pandas/plotting/_matplotlib/tools.py:307: MatplotlibDeprecationWarning: 
The colNum attribute was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use ax.get_subplotspec().colspan.start instead.
  layout[ax.rowNum, ax.colNum] = ax.get_visible()
/usr/local/lib/python3.6/dist-packages/pandas/plotting/_matplotlib/tools.py:313: MatplotlibDeprecationWarning: 
The rowNum attribute was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use ax.get_subplotspec().rowspan.start instead.
  if not layout[ax.rowNum + 1, ax.colNum]:
/usr/local/lib/python3.6/dist-packages/pandas/plotting/_matplotlib/tools.py:3

In [9]:
#display stats for the portfolio
stats = prices.calc_stats()
stats.display()

/usr/local/lib/python3.6/dist-packages/ffn/core.py:2056: RuntimeWarning: divide by zero encountered in true_divide
  res = np.divide(er.mean(), std)


Stat                 hsic        incy        msft        pki         etfc        cboe        intu        nbl         nlsn        lb
-------------------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------
Start                2011-01-27  2011-01-27  2011-01-27  2011-01-27  2011-01-27  2011-01-27  2011-01-27  2011-01-27  2011-01-27  2011-01-27
End                  2015-03-26  2015-03-26  2015-03-26  2015-03-26  2015-03-26  2015-03-26  2015-03-26  2015-03-26  2015-03-26  2015-03-26
Risk-free rate       0.00%       0.00%       0.00%       0.00%       0.00%       0.00%       0.00%       0.00%       0.00%       0.00%

Total Return         108.81%     483.39%     60.63%      101.33%     63.98%      169.75%     113.47%     19.11%      77.52%      294.40%
Daily Sharpe         1.03        1.11        0.62        0.79        0.50        1.11        0.92        0.29        0.70        1.41
Daily Sortino        1.68        2.06    

Calculate mean expected returns and covariance matrix for the price data (as per defaults specified in the PyPortfolioOpt docs https://pyportfolioopt.readthedocs.io/en/latest/UserGuide.html )

In [10]:
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage

mu = mean_historical_return(prices)
S = CovarianceShrinkage(prices).ledoit_wolf()
print("Mean expected returns: \n")
print(mu, "\n")
print("Covariance matrix: \n")
print(S, "\n")

Mean expected returns: 

hsic    0.195294
incy    0.543646
msft    0.140022
pki     0.201305
etfc    0.196786
cboe    0.268109
intu    0.208679
nbl     0.088353
nlsn    0.166498
lb      0.364036
dtype: float64 

Covariance matrix: 

          hsic      incy      msft  ...       nbl      nlsn        lb
hsic  0.036597  0.034487  0.019361  ...  0.027443  0.018020  0.022877
incy  0.034487  0.238139  0.025686  ...  0.043428  0.024293  0.036955
msft  0.019361  0.025686  0.052058  ...  0.027607  0.015826  0.021516
pki   0.027809  0.045353  0.022056  ...  0.038505  0.019186  0.027978
etfc  0.039880  0.065669  0.037999  ...  0.056510  0.032380  0.041508
cboe  0.019534  0.033653  0.017136  ...  0.021766  0.015316  0.019960
intu  0.023816  0.030230  0.023265  ...  0.028563  0.017917  0.023437
nbl   0.027443  0.043428  0.027607  ...  0.092231  0.027484  0.029484
nlsn  0.018020  0.024293  0.015826  ...  0.027484  0.057148  0.019922
lb    0.022877  0.036955  0.021516  ...  0.029484  0.019922  0.0669

Perform efficient frontier optimization using the max-sharpe ratio as the optimization parameter

In [11]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import objective_functions

#calculate efficient frontier
ef = EfficientFrontier(mu, S)

"""
use gamma parameter to reduce tendency of optimization to result in 0 asset weights, 
as this is detrimental to portfolio diversication

https://pyportfolioopt.readthedocs.io/en/latest/EfficientFrontier.html#l2-regularisation

"""
ef.add_objective(objective_functions.L2_reg, gamma=0.1)
weights = ef.max_sharpe()

#use cleaned weights function to round data
cleaned_weights = ef.clean_weights()

#ef.save_weights_to_file("weights.txt")  # saves to file
print(cleaned_weights)

#print expected performance
ef.portfolio_performance(verbose=True)

{'hsic': 0.08171, 'incy': 0.22219, 'msft': 0.01566, 'pki': 0.04396, 'etfc': 0.0, 'cboe': 0.18589, 'intu': 0.09897, 'nbl': 0.0, 'nlsn': 0.06474, 'lb': 0.28688}
Expected annual return: 33.3%
Annual volatility: 20.5%
Sharpe Ratio: 1.53


/usr/local/lib/python3.6/dist-packages/pypfopt/efficient_frontier.py:178: UserWarning: max_sharpe transforms the optimisation problem so additional objectives may not work as expected.
  "max_sharpe transforms the optimisation problem so additional objectives may not work as expected."


(0.33349813348700236, 0.20498732915454207, 1.5293537155686967)

Now that the weights have been generated for the portfolio, 
the weighted and unweighted portfolio performance can be tested over the period of 2015-2020. The performance of the two portfolios can be compared to determine if the optimization indeed will improve returns.

The annualized returns of the weighted portfolio can also be compared to the estimation provided by the PyPortfolioOpt library.

In [0]:
#obtain price data for test period
testPrices = pd.DataFrame()
testPrices = ffn.get(randomTickers,start='2015-03-26', end='2020-03-26')
ax = testPrices.rebase().plot(title="Performance of random portfolio over 2015-2020")

Now we can calulate annualized return, volatility and sharpe ratio for the unweighted and weighted portfolios

In [13]:
#https://towardsdatascience.com/optimizing-portfolios-with-modern-portfolio-theory-using-python-60ce9a597808 
print("Ticker Labels:")
print(randomTickers)
print("Equi-weighted portfolio weights:")
print(np.full(10,.1), "\n")
print("Optimized portfolio weights:")
print(list(cleaned_weights.values()), "\n")

opt_weights = list(cleaned_weights.values())
weights_arr = [np.full(10,.1), opt_weights]
returns_arr = []
volatility_arr = []

print(" --- Results --- ")
for weights in weights_arr:
  returns = prices.pct_change()
  
  # mean daily return and covariance of daily returns
  mean_daily_returns = returns.mean()
  cov_matrix = returns.cov()

  # portfolio weights
  weights = np.asarray(weights)
  
  portfolio_return = round(np.sum(mean_daily_returns * weights) * 252,2)
  portfolio_std_dev = round(np.sqrt(np.dot(weights.T,np.dot(cov_matrix, weights))) * np.sqrt(252),2)

  returns_arr.append(portfolio_return)
  volatility_arr.append(portfolio_std_dev)

  if(weights[0] == .1):
    print("Equal weighted portfolio stats:")
  else:
    print("Optimized portfolio stats:")
  print("Annualised return: " + str(portfolio_return))
  print("Volatility: " + str(portfolio_std_dev) + "\n")

print("Percent change in return after optimization: " + str(((returns_arr[1] - returns_arr[0])/returns_arr[0])))
print("Percent change in volatility after optimization: " + str(((volatility_arr[1] - volatility_arr[0])/volatility_arr[0])))

Ticker Labels:
['HSIC', 'INCY', 'MSFT', 'PKI', 'ETFC', 'CBOE', 'INTU', 'NBL', 'NLSN', 'LB']
Equi-weighted portfolio weights:
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1] 

Optimized portfolio weights:
[0.08171, 0.22219, 0.01566, 0.04396, 0.0, 0.18589, 0.09897, 0.0, 0.06474, 0.28688] 

 --- Results --- 
Equal weighted portfolio stats:
Annualised return: 0.24
Volatility: 0.19

Optimized portfolio stats:
Annualised return: 0.33
Volatility: 0.21

Percent change in return after optimization: 0.3750000000000001
Percent change in volatility after optimization: 0.1052631578947368


# **--- Optimization Results ---**

**Percent change in returns:** 58.8%

**Percent change in risk:** -5%

As expected, the portfolio optimization not only increased annualized returns, but also slightly decreased volitility.

Now we can graph the performance of the two portfolios with respect to the individual securities to visualize their performance.

In [14]:
#unweighted prices can be found by averaging the daily prices of all stocks in the portfolio
uw_portfolio = testPrices.sum(axis=1)
ax = uw_portfolio.plot(title="Performance of unweighted portfolio over 2015-2020")
#random_tickers_lower = [x.lower() for x in randomTickers]
#print(random_tickers_lower)

"""

Basically now I dont know how to properly multiply the opt_weights vector by row
with the price data in the dataframe. I've tried converting the data to a series first
(see here: https://stackoverflow.com/questions/13166842/pandas-dataframe-multiply-with-a-series)
but no luck.

"""

def multiplyWithPrice(row, weights):
  #print(row)
  #print(row.name)
  #print(row.values)
  #print(weights)
  #print(weights[row.name])
  modifiedRow = []
  for i in (0, len(row.values)-1):
    row.values[i] = row.values[i]*weights[i]
    #modifiedRow.append(row[i]*weights[row.name])
  #return modifiedRow
  return row

#print(cleaned_weights)
opt_weights_ser = pd.Series(opt_weights)
print(opt_weights_ser)
print("unweighted portfolio")
print(uw_portfolio)

#multiplyWithPrice(column, cleaned_weights)
temp = testPrices
w_portfolio = temp.apply(lambda row: multiplyWithPrice(row, opt_weights_ser), axis=1)
ax1 = testPrices.plot(title="Performance of individual stocks in portfolio before optimization over 2015-2020")
ax2 = w_portfolio.plot(title="Performance of individual stocks in portfolio after optimization over 2015-2020")

print("weighted portfolio before finding sum:")
print(w_portfolio)
w_portfolio = w_portfolio.sum(axis=1)

print("weighted portfolio")
print(w_portfolio)
ax3 = w_portfolio.plot(title="Performance of weighted portfolio after optimization over 2015-2020")




#first plot performance of two portfolios
portfolios = pd.concat([uw_portfolio, w_portfolio], axis=1, keys=['unweighted', 'optimized'])
print(portfolios)
ax4 = portfolios.plot(title="Performance of portfolio before and after optimization over 2015-2020")

#add portfolio data to price data
#testPrices['uw_portfolio'] = uw_portfolio
#testPrices['w_portfolio'] = w_portfolio

#plot new data
#ax = testPrices.plot(title="Performance of random portfolio over 2015-2020")



0    0.08171
1    0.22219
2    0.01566
3    0.04396
4    0.00000
5    0.18589
6    0.09897
7    0.00000
8    0.06474
9    0.28688
dtype: float64
unweighted portfolio
Date
2015-03-26    555.565128
2015-03-27    562.964359
2015-03-30    569.021252
2015-03-31    563.505457
2015-04-01    559.625160
                 ...    
2020-03-20    658.150001
2020-03-23    634.299997
2020-03-24    713.770003
2020-03-25    721.669996
2020-03-26    769.890008
Length: 1260, dtype: float64
weighted portfolio before finding sum:
                hsic       incy        msft  ...        nbl       nlsn         lb
Date                                         ...                                 
2015-03-26  4.397280  89.199997   37.158421  ...  44.213196  35.637016  20.862522
2015-03-27  4.447588  94.360001   36.942028  ...  44.268524  35.885479  21.067921
2015-03-30  4.512955  94.190002   36.933010  ...  45.236904  37.094627  21.262027
2015-03-31  4.473863  91.660004   36.662495  ...  45.098564  36.912437  21.2